In [1]:
#### This cell is to make spark work on a windows laptop
import os
import sys

# Path for spark source folder
os.environ['SPARK_HOME']="C:\spark-2.0.1-bin-hadoop2.7"

# Append pyspark  to Python Path
sys.path.append("C:\spark-2.0.1-bin-hadoop2.7\python")
sys.path.append("C:\spark-2.0.1-bin-hadoop2.7\python\lib\py4j-0.10.3-src.zip")
#os.environ['SPARK_EXECUTOR_MEMORY']="5G"

try:
    from pyspark import SparkContext
    from pyspark import SparkConf
    from pyspark.sql import SQLContext
    print ("Successfully imported Spark Modules")

except ImportError as e:
    print ("Can not import Spark Modules", e)
    sys.exit(1)

# Initialize SparkContext
sc = SparkContext('local')
words = sc.parallelize(["scala","java","hadoop","spark","akka"])
print (words.count())
print(words.countByValue())

Successfully imported Spark Modules
5
defaultdict(<class 'int'>, {'scala': 1, 'java': 1, 'spark': 1, 'akka': 1, 'hadoop': 1})


In [19]:
import pyspark
import pyspark.sql.types as T
import pyspark.sql.functions as F
import pyspark.ml
import pyspark.mllib.linalg
from pyspark.ml.linalg import Vectors,VectorUDT
sqlContext = SQLContext(sc)
data = [0, 3, 0, 4]
d = {}
d['DenseVector'] = Vectors.dense(data)
d['VectorUDT'] = pyspark.ml.linalg.SparseVector(4, dict(enumerate(data)))
d['old_DenseVector'] = pyspark.mllib.linalg.DenseVector(data)
d['SparseVector'] = pyspark.ml.linalg.SparseVector(4, dict(enumerate(data)))
d['old_SparseVector'] = pyspark.mllib.linalg.SparseVector(4, dict(enumerate(data)))
df = sqlContext.createDataFrame([d])
df.printSchema()

root
 |-- DenseVector: vector (nullable = true)
 |-- SparseVector: vector (nullable = true)
 |-- VectorUDT: vector (nullable = true)
 |-- old_DenseVector: vector (nullable = true)
 |-- old_SparseVector: vector (nullable = true)



C:\spark-2.0.1-bin-hadoop2.7\python\pyspark\sql\session.py:316: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [4]:
for x in df.first().asDict().items():
  print(x[0], type(x[1]))

DenseVector <class 'pyspark.ml.linalg.DenseVector'>
old_SparseVector <class 'pyspark.mllib.linalg.SparseVector'>
old_DenseVector <class 'pyspark.mllib.linalg.DenseVector'>
SparseVector <class 'pyspark.ml.linalg.SparseVector'>


In [20]:
from pyspark.ml.linalg import Vectors
df = sqlContext.createDataFrame([
(0.0, Vectors.dense([0.0, 0.0,0.0, 0.0])),
(1.0, Vectors.dense([0.0, 1.0,1.0, 1.0])),
(1.0, Vectors.dense([1.0, 0.0,0.0, 0.0])),
(0.0, Vectors.dense([1.0, 1.0,1.0, 1.0]))], ["label", "features"])
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4, 2, 4], blockSize=1, seed=123)
model = mlp.fit(df)

testDF = sqlContext.createDataFrame([
(Vectors.dense([1.0, 0.0,0.0, 0.0]),),
(Vectors.dense([1.0, 1.0,1.0, 1.0]),)], ["features"])
model.transform(testDF).show()

NameError: name 'MultilayerPerceptronClassifier' is not defined

In [ ]:
from pyspark.ml.linalg import Vectors,VectorUDT
import pyspark.ml.linalg as ml
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.linalg import SparseVector, VectorUDT
from pyspark.sql import types

def traitement(x):
    return (float(x[-1]-mean),SparseVector(keep_index(x)))
    
list_indices = [i for i in range(2,20)]

schema = types.StructType([
    types.StructField("label", types.DoubleType(), True),
    types.StructField("features", VectorUDT(), True)
])

df_Perceptron_tuple = sqlContext.createDataFrame(df.rdd.map(lambda x: traitement(x)),schema)
(df_Perceptron_tuple_train,df_Perceptron_tuple_test) = df_Perceptron_tuple.randomSplit([0.7, 0.3])
df_Perceptron_tuple_train.select("features").printSchema()
   
layers = [len(list_indices), 5, 4, 5]
# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
# train the model
model = trainer.fit(df_Perceptron_tuple_train)
# compute accuracy on the test set
result = model.transform(df_Perceptron_tuple_test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Accuracy: " + str(evaluator.evaluate(predictionAndLabels)))